# Jérôme II – Object Detection Pipeline with YOLOv8

**Author:** Éloïse Delerue

**Laboratory:** Université Paris Sciences & Lettres

**Contact:** eloise.delerue@psl.eu

**Date:** November 7th, 2025

## 0. Read Me

### 0.1. Overview

**Jérôme II** is a demonstration notebook implementing a full image processing and object detection pipeline using a YOLOv8 model from [Ultralytics](https://github.com/ultralytics/ultralytics). It was designed to be a follow-up to **Jérôme I** (YOLOv8 fine-tuning).

This notebook loads a dataset of images, performs inference with a trained model, visualises detected objects, and optionally exports results for further analysis.

**Please note that it is designed to run in Google Colab.**

### 0.2. Data

The dataset must be provided as a **ZIP archive containing images with the following structure**:

In [ ]:
dataset.zip
 ├── image_001.jpg
 ├── image_002.jpg
 ├── ...

The dataset will be automatically extracted in the notebook environment before inference.

**Please note that a zipped dataset is provided with this notebook, titled "bacchus", for inference; but any other properly structured dataset will do the job.**

### 0.3. Model

The notebook currently uses the YOLOv8n pre-trained model **for demonstration purposes**. You may replace yolov8n.pt with any fine-tuned YOLO model (e.g., yolov8m.pt, yolov8l.pt, or a custom checkpoint). The main objective would be to use a model resulting from the fine-tuning performed in **Jérôme I**.

### 0.4. Inference

Detection results are displayed inline using OpenCV’s cv2_imshow() function.
Bounding boxes, labels, and confidence scores are drawn on each processed image for clarity.

### 0.5. Results Export

The final cell allows exporting the processed images and detection results into a ZIP file.

To enable export, simply **uncomment the corresponding cell in the notebook**.

## 1. Setup & Libraries Import

In [1]:
# Installing Required Librarie
!pip install -q ultralytics opencv-python matplotlib pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00


In [6]:
# Importing Libraries for File and Directory Management
import os
import zipfile

# Importing Libraries for Google Colab Integration
from google.colab import files
from google.colab.patches import cv2_imshow

# Importing Libraries for Computer Vision
import cv2

# Importing Ultralytics YOLO for Object Detection
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## 2. Helper Functions

### 2.1. Visualisation Functions

In [7]:
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    """
    Draws a bounding box and label on an image.

    Args:
        image: The image to draw on.
        box: Bounding box coordinates as [x1, y1, x2, y2].
        label: Label to display.
        color: Color of the bounding box.
        txt_color: Color of the label text.
    """
    lw = max(round(sum(image.shape) / 2 * 0.003), 2)
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    cv2.rectangle(image, p1, p2, color, thickness=lw, lineType=cv2.LINE_AA)
    if label:
        tf = max(lw - 1, 1)
        w, h = cv2.getTextSize(label, 0, fontScale=lw / 3, thickness=tf)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image, label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, lw / 3, txt_color, thickness=tf, lineType=cv2.LINE_AA)

In [8]:
def plot_bboxes(image, boxes, labels=[], colors=[], score=True, conf=None):
    """
    Plots bounding boxes on an image.

    Args:
        image: The image to plot on.
        boxes: List of bounding boxes.
        labels: Dictionary mapping class IDs to labels.
        colors: List of colors for each class.
        score: Whether to display confidence scores.
        conf: Confidence threshold for filtering boxes.
    """
    if not labels:
        labels = {0: 'person'}
    if not colors:
        colors = [(89, 161, 197), (67, 161, 255), (19, 222, 24), (186, 55, 2), (167, 146, 11)]
    for box in boxes:
        x1, y1, x2, y2, confidence, class_id = box
        class_id = int(class_id)
        if conf and confidence < conf:
            continue
        label = f"{labels.get(class_id, f'class{class_id}')} {confidence*100:.1f}%" if score else labels.get(class_id, f'class{class_id}')
        color = colors[class_id % len(colors)]
        box_label(image, [x1, y1, x2, y2], label, color)

### 2.2. Object Detection Function

In [9]:
def detect_motifs(model, image_path, confidence_threshold=0.5):
    """
    Detects objects in an image using a YOLO model.

    Args:
        model: The YOLO model.
        image_path: Path to the image.
        confidence_threshold: Minimum confidence for detections.

    Returns:
        List of detections, each as [x1, y1, x2, y2, confidence, class_id].
    """
    results = model(image_path, conf=confidence_threshold)
    detections = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = float(box.conf[0])
            class_id = int(box.cls[0])
            detections.append([x1, y1, x2, y2, confidence, class_id])
    return detections

## 3. Model Loading

In [10]:
# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

# Or, if you have your own model file:

# uploaded = files.upload()

# # Get the filename of the uploaded model
# model_path = list(uploaded.keys())[0]

# # Load your custom model
# model = YOLO(model_path)

## 4. Images Loading & Extraction

In [11]:
# Upload and extract images
uploaded = files.upload()
uploaded_files = list(uploaded.keys())

Saving bacchus.zip to bacchus.zip


In [12]:
# Extract images if a ZIP file is uploaded
if uploaded_files and uploaded_files[0].endswith('.zip'):
    zip_path = uploaded_files[0]
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("uploaded_images")
    extracted_images = [f"uploaded_images/{f}" for f in os.listdir("uploaded_images")
                        if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
else:
    extracted_images = uploaded_files

## 5. Images Processing & Pattern Detection

In [13]:
# Create results directory
os.makedirs("results", exist_ok=True)

In [14]:
# List to store images with "person" detections and their bounding boxes
person_images = []

In [15]:
# Process each image
for image_path in extracted_images:
    image = cv2.imread(image_path)
    detections = detect_motifs(model, image_path, confidence_threshold=0.5)
    person_detections = [box for box in detections if int(box[5]) == 0]  # class_id 0 is 'person'

    if person_detections:
        plot_bboxes(image, person_detections)
        result_path = f"results/{os.path.basename(image_path)}"
        cv2.imwrite(result_path, image)
        print(f"Image: {image_path}")
        for box in person_detections:
            x1, y1, x2, y2, confidence, _ = box
            print(f"  Bounding box: ({x1}, {y1}, {x2}, {y2}), Confidence: {confidence:.2f}")
        person_images.append({
            "image": image_path,
            "bounding_boxes": [{"coordinates": (x1, y1, x2, y2), "confidence": confidence} for x1, y1, x2, y2, confidence, _ in person_detections]
        })
    else:
        print(f"No person detected in {image_path}")


image 1/1 /content/uploaded_images/04.jpg: 448x640 1 person, 368.3ms
Speed: 16.1ms preprocess, 368.3ms inference, 29.5ms postprocess per image at shape (1, 3, 448, 640)
Image: uploaded_images/04.jpg
  Bounding box: (645, 161, 763, 453), Confidence: 0.58

image 1/1 /content/uploaded_images/11.jpg: 448x640 3 persons, 183.9ms
Speed: 4.1ms preprocess, 183.9ms inference, 3.5ms postprocess per image at shape (1, 3, 448, 640)
Image: uploaded_images/11.jpg
  Bounding box: (573, 85, 811, 705), Confidence: 0.84
  Bounding box: (43, 238, 275, 699), Confidence: 0.82
  Bounding box: (442, 398, 567, 738), Confidence: 0.66

image 1/1 /content/uploaded_images/53.jpg: 416x640 1 person, 2 dogs, 163.3ms
Speed: 2.6ms preprocess, 163.3ms inference, 1.6ms postprocess per image at shape (1, 3, 416, 640)
Image: uploaded_images/53.jpg
  Bounding box: (321, 7, 633, 415), Confidence: 0.63

image 1/1 /content/uploaded_images/48.jpg: 256x640 3 persons, 3 dogs, 110.1ms
Speed: 1.8ms preprocess, 110.1ms inference, 1

## 6. Processed Images Display

In [ ]:
# Loop through each image path in the extracted_images list
for image_path in extracted_images:
    result_path = f"results/{os.path.basename(image_path)}"
    if os.path.exists(result_path):
        image = cv2.imread(result_path)
        cv2_imshow(image)

## 7. Results Packaging and Downloading

In [17]:
# Zip the results directory for download
!zip -r results.zip results/

  adding: results/ (stored 0%)
  adding: results/04.jpg (deflated 4%)
  adding: results/11.jpg (deflated 4%)
  adding: results/53.jpg (deflated 7%)
  adding: results/48.jpg (deflated 6%)
  adding: results/25.jpg (deflated 4%)
  adding: results/41.jpg (deflated 4%)
  adding: results/55.jpg (deflated 5%)
  adding: results/23.jpg (deflated 6%)
  adding: results/29.jpg (deflated 4%)
  adding: results/30.jpg (deflated 5%)
  adding: results/08.jpg (deflated 6%)
  adding: results/43.jpg (deflated 4%)
  adding: results/54.jpg (deflated 4%)
  adding: results/17.jpg (deflated 13%)
  adding: results/50.jpg (deflated 3%)
  adding: results/31.jpg (deflated 7%)
  adding: results/56.jpg (deflated 7%)
  adding: results/06.jpg (deflated 6%)
  adding: results/51.jpg (deflated 6%)
  adding: results/16.jpg (deflated 6%)
  adding: results/52.jpg (deflated 0%)
  adding: results/35.jpg (deflated 4%)
  adding: results/45.jpg (deflated 6%)
  adding: results/21.jpg (deflated 5%)
  adding: results/49.jpg (deflat

In [18]:
# Download the zip file
# files.download("results.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>